In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
import shutil
from scipy.stats import norm
from PIL import Image
import numpy as np
import pandas as pd
from pad.detector import PAD
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


In [ ]:
from scipy.stats import norm
import numpy as np

class Utils():
  def get_norm_dist(self, scores):
    mu, std = norm.fit(scores)
    self.nrm = norm(mu, std)
    x = np.linspace(scores.min(), scores.max(), 100)
    p = self.nrm.pdf(x)
    return self.nrm, p, x, mu, std

  def is_accepted(self, score, max):
    rand = np.random.uniform(0, max, 1)[0]
    pdf_val = self.nrm.pdf(score)
    if pdf_val > rand:
      return True
    return False


In [ ]:
from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

class Generator():
  def __init__(self, model_path):
    self.generator = load_model(model_path)

  def generate_images(self, image_count):
    images = []
    seed = tf.random.normal([image_count, 128])
    images = self.generator(seed, training=False)
    return [np.array(image* 127.5 + 127.5).astype(np.uint8) for image in images]

  def show_image(self, image):
    plt.imshow(image)
    plt.axis('off')

In [ ]:
class PadHelper(PAD):
  fig_dir = "./data/figures/pad_distribution"
  synthetic_dir = "./data/synthetic"
  output_file = "./data/info/append.json"
  utils = Utils()

  def __init__(self, model_path, generator, train_pad_scores):
    super().__init__(model_path)
    self.generator = generator
    self.train_pad_scores = train_pad_scores

  def sample(self, fold, count):
    _, p, x, mu, std = self.utils.get_norm_dist(self.train_pad_scores)
    sample_list = []
    dir_len = self.get_fold_generated_count(fold) + 1
    generated_images = self.generator.generate_images(count)
    for idx, image in enumerate(generated_images):
      image = Image.fromarray(image.astype('uint8'), 'RGB')
      pad_score = self.get_pad_score_from_image(image);
      accept = self.utils.is_accepted(pad_score, p.max())
      count = idx + dir_len
      filename = f"{fold}_{count}.jpg"
      dict={"fold": fold, "image": filename, "pad_score": pad_score, "pad_status": accept, "train_match_scores": None, "verieye_status":None}
      sample_list.append(dict)
      ##image.save(os.path.join(self.synthetic_dir,filename))
    df = pd.DataFrame(sample_list)
    if not os.path.exists(self.output_file):
      df.to_json(self.output_file, orient="records")
    else:
      self.append_to_json(self.output_file, df)
    return df

  def get_fold_generated_count(self, fold):
    if not os.path.exists(self.synthetic_dir):
      return 0
    dir_list = os.listdir(self.synthetic_dir)
    fold_count = [filename for filename in dir_list if filename.split('_')[0] == str(fold)]
    return len(fold_count)

  def append_to_json(self, jsonFile, newData):
    df = pd.read_json(jsonFile, orient="records")
    df = pd.concat([df, newData], ignore_index=True)
    df.to_json(jsonFile, orient="records")

In [ ]:
def create_dirs():
    if os.path.exists(PadHelper.synthetic_dir):
      shutil.rmtree(PadHelper.synthetic_dir)
    if os.path.exists(PadHelper.output_file):
      os.remove(PadHelper.output_file)
    if os.path.exists(PadHelper.fig_dir):
      shutil.rmtree(PadHelper.fig_dir)
    os.mkdir(PadHelper.synthetic_dir)
    os.mkdir(PadHelper.fig_dir)

In [ ]:
sample_count = 2000
train_df = pd.read_csv("./data/info/train.csv", sep=";")
for fold in range (1,2):
  train_pad_scores = np.array(train_df[train_df["group"] != fold]["pad_score"])
  generator = Generator(f"./models/{str(fold)}.h5")
  pad_helper = PadHelper("./pad/Model/D-NetPAD_Model.pth", generator, train_pad_scores)
  df = pad_helper.sample(fold, sample_count)

In [12]:
df = pd.read_json("./data/info/synthetic.json", orient="records")
train_df = pd.read_csv("./data/info/train.csv", sep=";")
utils = Utils()
for fold in range(1,6):
  _, p, x, mu, std = utils.get_norm_dist(train_df[train_df["group"] != fold]["pad_score"])
  for idx, row in df[df["fold"] == fold].iterrows():
    df.loc[idx, "pad_status"] = utils.is_accepted(row["pad_score"], p.max())
df.to_json("./data/info/pad_fixed.json", orient="records")